# Data Processing
***

#### Importing packages and setting up the notebook

In [68]:
#Importing Packages
import sqlite3
import pandas as pd
import numpy as np
from EnhancedPandas import EnhancedPandas as ep
from datetime import datetime

#Setting the default max number of lines of cell output to None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Connecting to the Database
con = sqlite3.connect('C:/Users/Dell/OneDrive/Dokumenty/Data Engineering/Codecademy Course/subscriber_cancellation_pipeline/subscriber-pipeline/subscriber_cancellation_pipeline/databases/cademycode.db')
cur = con.cursor()

#Creating DataFrames
students_df = pd.read_sql_query("SELECT * FROM cademycode_students", con)
paths_df = pd.read_sql_query("SELECT * FROM cademycode_courses", con)
jobs_df = pd.read_sql_query("SELECT * FROM cademycode_student_jobs", con)
con.close() 

#### Defining Custom Exception

In [69]:
class DataTypeError(Exception):
    def __str__(self):
        return('A float value is present in a column meant to have only integers')

This notebook uses standard Python libraries and packages as well as custom-made EnhancedPandas library written to speed up the process of data cleaning

## Cleaning The Students DataFrame

### Basic Checks

In [70]:
students_df.head(5)

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64


#### Splitting the contact_info column into address and email columns

In [71]:
#Splitting the column
def get_email(x):
    return (x.split(':'))[2][2:-2]

def get_address(x):
    return (x.split(':'))[1][2:-10]

students_df['email']= list(map(get_email, students_df['contact_info']))
students_df['address'] = list(map(get_address, students_df['contact_info']))

#Dropping the original column
students_df.drop(columns=['contact_info'], inplace=True)
students_df.head(5)

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,address
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,annabelle_avery9376@woohoo.com,"303 N Timber Key, Irondale, Wisconsin, 84736"
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,rubio6772@hmail.com,"767 Crescent Fair, Shoals, Indiana, 37439"
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,hosea_dale8084@coldmail.com,"P.O. Box 41269, St. Bonaventure, Virginia, 83637"
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,kirk4005@hmail.com,"517 SE Wintergreen Isle, Lane, Arkansas, 82242"
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,alexander9810@hmail.com,"18 Cinder Cliff, Doyles borough, Rhode Island,..."


#### Splitting the address column into individual pieces

In [72]:
address_list = students_df['address'].to_list()
streets = []
cities = []
states = []
postcodes = []

for address in address_list:
    address_split = address.split(',')
    street = address_split[0]
    city = address_split[1][1:]
    state = address_split[2][1:]
    postcode = address_split[3][1:]
    streets.append(street)
    cities.append(city)
    states.append(state)
    postcodes.append(postcode)

students_df['street'] = streets
students_df['city'] = cities
students_df['state'] = states
students_df['postcode'] = postcodes

students_df = students_df.drop('address', axis = 1)
students_df.head(5)

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,street,city,state,postcode
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,rubio6772@hmail.com,767 Crescent Fair,Shoals,Indiana,37439
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,hosea_dale8084@coldmail.com,P.O. Box 41269,St. Bonaventure,Virginia,83637
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,kirk4005@hmail.com,517 SE Wintergreen Isle,Lane,Arkansas,82242
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,alexander9810@hmail.com,18 Cinder Cliff,Doyles borough,Rhode Island,73737


#### Checking the Data Types

In [73]:
students_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   job_id                  4995 non-null   object
 5   num_course_taken        4749 non-null   object
 6   current_career_path_id  4529 non-null   object
 7   time_spent_hrs          4529 non-null   object
 8   email                   5000 non-null   object
 9   street                  5000 non-null   object
 10  city                    5000 non-null   object
 11  state                   5000 non-null   object
 12  postcode                5000 non-null   object
dtypes: int64(1), object(12)
memory usage: 507.9+ KB


#### Converting Data Types

In [74]:
students_df['dob'] = pd.to_datetime(students_df['dob'])
students_df['sex'] = students_df['sex'].astype(str)
students_df['time_spent_hrs'] = pd.to_numeric(students_df['time_spent_hrs'])
students_df['email'] = students_df['email'].astype(str)
students_df['postcode'] = students_df['postcode'].astype(int)
students_df['job_id'] = pd.to_numeric(students_df['job_id'])
students_df['num_course_taken'] = pd.to_numeric(students_df['num_course_taken'])
students_df['current_career_path_id'] = pd.to_numeric(students_df['current_career_path_id'])
print(students_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   sex                     5000 non-null   object        
 4   job_id                  4995 non-null   float64       
 5   num_course_taken        4749 non-null   float64       
 6   current_career_path_id  4529 non-null   float64       
 7   time_spent_hrs          4529 non-null   float64       
 8   email                   5000 non-null   object        
 9   street                  5000 non-null   object        
 10  city                    5000 non-null   object        
 11  state                   5000 non-null   object        
 12  postcode                5000 non-null   int32   

#### Adding Age Column for easier analysis

In [75]:
students_dobs = students_df['dob'].to_list()
students_ages = []

for student_dob in students_dobs:
    student_age = round((datetime.today() - student_dob).total_seconds() / (365 * 24 * 3600))
    students_ages.append(student_age)

students_df['age'] = students_ages
students_df.head(1)

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,street,city,state,postcode,age
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736,80


#### Checking for duplicated Values in the Dataset

In [76]:
print(students_df.duplicated().sum())

0


#### Checking for Sanity and numerical Outliers

In [77]:
students_df.describe()

,uuid,job_id,num_course_taken,current_career_path_id,time_spent_hrs,postcode,age
count,5000.000000,4995.000000,4749.000000,4529.000000,4529.000000,5000.000000,5000.000000
mean,2500.500000,4.172172,7.533586,5.478914,11.520625,49808.924000,50.774200
std,1443.520003,2.148098,4.609884,2.874135,7.564228,28797.817944,18.336828
min,1.000000,1.000000,0.000000,1.000000,0.000000,37.000000,19.000000
25%,1250.750000,2.000000,4.000000,3.000000,5.380000,25228.500000,35.000000
50%,2500.500000,4.000000,7.000000,5.000000,10.670000,49768.000000,51.000000
75%,3750.250000,6.000000,12.000000,8.000000,16.750000,74737.000000,67.000000
max,5000.000000,8.000000,15.000000,10.000000,35.980000,99984.000000,82.000000


No visible outliers are in the dataset. It is interesting that the minimum values of num_course_taken and time_spent_hrs is 0. The maximum and minimum values of the various columns will be useful for checking referential integrity later on. 

### Missing Data

#### Overall DataFrame Check

In [78]:
print(ep.viewna_all(students_df))

                    column  missing count  percentage missing
0                     uuid              0                0.00
1                     name              0                0.00
2                      dob              0                0.00
3                      sex              0                0.00
4                   job_id              5                0.10
5         num_course_taken            251                5.02
6   current_career_path_id            471                9.42
7           time_spent_hrs            471                9.42
8                    email              0                0.00
9                   street              0                0.00
10                    city              0                0.00
11                   state              0                0.00
12                postcode              0                0.00
13                     age              0                0.00


 #### missing data in the job_id column

In [79]:
job_id_null_df = students_df[students_df['job_id'].isnull() == True]
job_id_null_df.head(10)

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,street,city,state,postcode,age
162,163,Glen Riley,2002-08-22,M,NaN,8.0,3.0,5.70,glen_riley4484@hmail.com,P.O. Box 37267,Cornlea village,Tennessee,19192,21
757,758,Mercedez Vorberg,2002-03-25,F,NaN,15.0,4.0,4.14,mercedez6297@woohoo.com,284 Cedar Seventh,Virden village,Washington,60489,22
854,855,Kurt Ho,2002-05-29,M,NaN,0.0,8.0,23.72,ho6107@inlook.com,P.O. Box 27254,Olin,New Hampshire,60067,22
1029,1030,Penny Gaines,2002-03-01,N,NaN,15.0,4.0,16.25,gaines2897@hmail.com,138 Misty Vale,Stockton borough,West Virginia,53630,22
1542,1543,Frederick Reilly,2002-11-13,M,NaN,7.0,9.0,21.32,frederick_reilly6971@woohoo.com,P.O. Box 40769,Quakervillage,Maryland,96218,21


Since there is so little of job_id data missing, it can be investigated by simply printing out the rows with the empty values. There is no visible pattern in this missing data, although the dob of all the students is similar and indicates young people. (The year of cancellation is unknown so the exact age can't be calculated.) It is possible that these people are students looking for job or are just unemployed. The option 'unemployed' should be added to the jobs table. This data is dropped and moved to a different DataFrame for the analytical team. 

In [80]:
students_df_job_id_null = job_id_null_df
students_df = students_df.dropna(subset='job_id')

#### Investigating missing data in the num_course_taken column

In [81]:
students_df[students_df['num_course_taken'].isnull() == True].reset_index().head(15)

,index,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,email,street,city,state,postcode,age
0,25,26,Doug Browning,1970-06-08,M,7.0,NaN,5.0,1.92,doug7761@inlook.com,P.O. Box 15845,Devine,Florida,23097,54
1,26,27,Damon Schrauwen,1953-10-31,M,4.0,NaN,10.0,3.73,damon9864@woohoo.com,P.O. Box 84659,Maben,Georgia,66137,70
2,51,52,Alisa Neil,1977-05-28,F,5.0,NaN,8.0,22.86,alisa9616@inlook.com,16 View Annex,Mosses,North Dakota,25748,47
3,70,71,Chauncey Hooper,1962-04-07,M,3.0,NaN,3.0,3.97,chauncey6352@woohoo.com,955 Dewy Flat,Slaughterville,South Carolina,22167,62
4,80,81,Ellyn van Heest,1984-06-28,F,3.0,NaN,10.0,12.39,ellyn_vanheest8375@hmail.com,872 Cider Glade,Chicken,Delaware,42689,39
5,96,97,Moises Krekel,2004-06-15,M,8.0,NaN,NaN,NaN,moises_krekel5984@woohoo.com,455 Silent Smith,Radium Springs,North Carolina,29278,19
6,139,140,Charlie Semvoort,2004-03-06,N,8.0,NaN,5.0,5.76,charlie8186@inlook.com,76 Dewy Quail,Whitesburg,South Carolina,24297,20
7,149,150,Lindsey Mcclain,1972-11-30,N,6.0,NaN,9.0,21.79,lindsey_mcclain8692@woohoo.com,751 Robin Highlands,Hurlock,Tennessee,33406,51
8,157,158,Ching Rijkensz,1988-02-17,N,3.0,NaN,7.0,2.36,ching_rijkensz6414@hmail.com,471 Timber Arbor,Oaks,Texas,84192,36
9,202,203,Bettie Torres,1973-06-05,N,6.0,NaN,9.0,6.04,bettie_torres6037@hmail.com,457 SW Valley,New Madison village,South Carolina,78280,51


At first sight there is no visible pattern of the missingness. It is strange that the num_course_taken column is empty even though there are values in the current_career_path_id and time_spent_hrs columns. However this can't be investigated further as it is not clear what does the num_course_taken column actually mean. 

##### Investigating the correlation between job_id and missing data in num_course_taken

In [82]:
ep.viewna_column(students_df, 'num_course_taken', 'job_id').head(15)

,job_id,num_course_taken number of records,num_course_taken missing values,Percentage Missing
0,1.0,693,30,4.329004
1,2.0,706,35,4.957507
2,3.0,675,31,4.592593
3,4.0,671,39,5.812221
4,5.0,660,32,4.848485
5,6.0,657,36,5.479452
6,7.0,680,36,5.294118
7,8.0,253,12,4.743083


no correlation

##### Investigating the correlation between current_career_path_id and num_course_taken

In [83]:
ep.viewna_column(students_df, 'num_course_taken', 'current_career_path_id').head(15)

,current_career_path_id,num_course_taken number of records,num_course_taken missing values,Percentage Missing
0,1.0,459,22,4.793028
1,2.0,450,22,4.888889
2,3.0,468,26,5.555556
3,4.0,432,31,7.175926
4,5.0,476,30,6.302521
5,6.0,454,22,4.845815
6,7.0,449,16,3.563474
7,8.0,436,16,3.669725
8,9.0,440,23,5.227273
9,10.0,460,23,5.000000


no correlation. Checking for other correlations would make no logical sense, therefore this missing data is classified as MCAR data and since it takes only 5% of the column, the rows including empty data are simply dropped. and moved to a separate DataFrame

In [84]:
students_df_num_course_taken_null = students_df[students_df['num_course_taken'].isnull() == True]
students_df = students_df.dropna(subset = 'num_course_taken')

#### Investigating missing data in the current_career_path_id and time_spent_hrs column 

The number of missing values in the current_career_path_id and time_spent_hrs columns is the same. The following cell verifies that there is one-to-one relationship with the missing data in these two columns.   

In [85]:
ep.viewna_column(students_df, 'time_spent_hrs', 'current_career_path_id').head(15)

,current_career_path_id,time_spent_hrs number of records,time_spent_hrs missing values,Percentage Missing
0,1.0,437,0,0.0
1,2.0,428,0,0.0
2,3.0,442,0,0.0
3,4.0,401,0,0.0
4,5.0,446,0,0.0
5,6.0,432,0,0.0
6,7.0,433,0,0.0
7,8.0,420,0,0.0
8,9.0,417,0,0.0
9,10.0,437,0,0.0


This means that the data is structurally missing and it also means that the time_spent_hrs counts only when the user is enrolled in a career path. In the time_spent_hrs column, 0 is therefore inputted instead of the null values and the null values in the current_career_path id will be replaced with a separate path_id indicating that the student has not chosen a career path. 

In [86]:
students_df = students_df.fillna(value = {'time_spent_hrs' : 0})
students_df = students_df.fillna(value = {'current_career_path_id' : 999})

In [87]:
ep.viewna_all(students_df).head(15)

,column,missing count,percentage missing
0,uuid,0,0.0
1,name,0,0.0
2,dob,0,0.0
3,sex,0,0.0
4,job_id,0,0.0
5,num_course_taken,0,0.0
6,current_career_path_id,0,0.0
7,time_spent_hrs,0,0.0
8,email,0,0.0
9,street,0,0.0


No missing data in the students table

### Converting float columns of the students dataframe to ints.

In the cells below, the columns whose data type is object are checked if unexpected values aren't included in them. These columns are name, email, sex, street, city, state

In [88]:
def to_int_converted(column):
    students_df[column] = pd.to_numeric(students_df[column])
    col_to_list = students_df[column].to_list()
    integers = []

    for i in range(len(col_to_list)):
        num = col_to_list[i]
        if num % 1 == 0:
            num = str(num)
            new_num = num.split('.')[0]
            new_num = int(new_num)
            integers.append(new_num)
        else:
            raise DataTypeError
    
    return integers

students_df['job_id'] = to_int_converted('job_id')
students_df['num_course_taken'] = to_int_converted('num_course_taken')
students_df['current_career_path_id'] = to_int_converted('current_career_path_id')

print(students_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4744 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4744 non-null   int64         
 1   name                    4744 non-null   object        
 2   dob                     4744 non-null   datetime64[ns]
 3   sex                     4744 non-null   object        
 4   job_id                  4744 non-null   int64         
 5   num_course_taken        4744 non-null   int64         
 6   current_career_path_id  4744 non-null   int64         
 7   time_spent_hrs          4744 non-null   float64       
 8   email                   4744 non-null   object        
 9   street                  4744 non-null   object        
 10  city                    4744 non-null   object        
 11  state                   4744 non-null   object        
 12  postcode                4744 non-null   int32   

## Cleaning Paths DataFrame

Checking the values and columns

In [89]:
paths_df.head(15)

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15


Checking the data types

In [90]:
paths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes


Checking for duplicates

In [91]:
print(paths_df.duplicated().sum())

0


The dataset is completely clean. The cell below adds the 'not selected' option to the dataframe as was mentioned above.

In [92]:
new_row = {'career_path_id' : 999, 'career_path_name' : 'not selected', 'hours_to_complete' : np.nan}
new_df = pd.DataFrame(new_row, index = [0])
paths_df = pd.concat([paths_df, new_df], ignore_index=True)
paths_df.head(15)

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20.0
1,2,data engineer,20.0
2,3,data analyst,12.0
3,4,software engineering,25.0
4,5,backend engineer,18.0
5,6,frontend engineer,20.0
6,7,iOS developer,27.0
7,8,android developer,27.0
8,9,machine learning engineer,35.0
9,10,ux/ui designer,15.0


## Cleaning Jobs DataFrame

Checking the values and columns

In [93]:
jobs_df.head(15)

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


Checking the data types

In [94]:
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        13 non-null     int64 
 1   job_category  13 non-null     object
 2   avg_salary    13 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 444.0+ bytes


In [95]:
print(jobs_df.duplicated().sum())
jobs_df = jobs_df.drop_duplicates()
jobs_df.head(15)

3


,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


The dataset is completely clean

### Reordering the students_df dataframe

In [96]:
students_df = students_df[['uuid', 'name', 'dob', 'age', 'sex', 'email', 'street', 'city', 'state', 'postcode', 'job_id', 'num_course_taken', 'current_career_path_id', 'time_spent_hrs']]
students_df.head(1)

,uuid,name,dob,age,sex,email,street,city,state,postcode,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,80,F,annabelle_avery9376@woohoo.com,303 N Timber Key,Irondale,Wisconsin,84736,7,6,1,4.99
